In [105]:
import Atlas
import Atlas.visitors as Visitors
import pandas as pd

In [106]:
try:
    tape = Atlas.Tape()
except:
    pass

In [107]:
evalDate = Atlas.Date(1, Atlas.August, 2020)
store = Atlas.MarketStore(evalDate, Atlas.CLP()) # store with CLP as base currency

# define curve
curveDayCounter = Atlas.Actual360()
curveCompounding = Atlas.Simple
curveFrequency = Atlas.Annual

clpRate = Atlas.Dual(0.03)
usdRate = Atlas.Dual(0.01)
fxSpot = 800
fx = Atlas.Dual(fxSpot)

# we need to register the input to the tape for later use
tape.registerInput(clpRate) 
tape.registerInput(usdRate) 
tape.registerInput(fx) 
tape.newRecording() # start recording, for later use


# add CLP Curve
strategy = Atlas.FlatForwardStrategy(evalDate, clpRate, curveDayCounter, curveCompounding, curveFrequency)
clpCurve = Atlas.YieldTermStructure(strategy)
index = Atlas.RateIndex(evalDate, curveFrequency, curveDayCounter, curveFrequency, curveCompounding)
store.addCurve("CLP", clpCurve, index, Atlas.CLP())

# add USD Curve
strategy = Atlas.FlatForwardStrategy(evalDate, usdRate, curveDayCounter, curveCompounding, curveFrequency)
usdCurve = Atlas.YieldTermStructure(strategy)
store.addCurve("USD", usdCurve, index, Atlas.USD())

# add FX
store.addExchangeRate(Atlas.CLP(), Atlas.USD(), fx)

In [108]:
#define interest rate
rateValue = Atlas.Dual(0.05)
dayCounter = Atlas.Thirty360()
compounding = Atlas.Simple
frequency = Atlas.Annual

rate = Atlas.InterestRate(rateValue, dayCounter, compounding, frequency)
discountContext = store.curveContext("CLP")
# define zero coupon instrument
notional = fxSpot
startDate = evalDate
endDate = Atlas.Date(1, Atlas.August, 2025)
paymentFrequency = Atlas.Semiannual
instrument = Atlas.FixedRateBulletInstrument(startDate, endDate, paymentFrequency, notional, rate, discountContext)

In [109]:
newIndexer = Visitors.IndexingVisitor()
newIndexer(instrument)
request  = newIndexer.getResults()

model = Atlas.SpotMarketDataModel(request, store) ## maybe add a setRequest method
marketData = model.marketData(evalDate)

newNPVVisitor = Visitors.NPVConstVisitor(marketData)
newNPVVisitor.visit(instrument)

newNPVVisitor.getResults()

Dual(879.192194)

In [110]:
for fx in marketData.fxs:
    print(fx)

print("")
for df in marketData.dfs:
    print(df)

Dual(1.000000)

Dual(1.000000)
Dual(0.984898)
Dual(0.970481)
Dual(0.956251)
Dual(0.942655)
Dual(0.929224)
Dual(0.916380)
Dual(0.903683)
Dual(0.891464)
Dual(0.879443)
Dual(0.867930)


In [111]:
newNotional = 1
instrument = Atlas.FixedRateBulletInstrument(startDate, endDate, paymentFrequency, newNotional, rate, discountContext)


instrument.currency(Atlas.USD())
instrument.discountCurveContext(store.curveContext("USD"))
instrument.applyCcy(False)


newIndexer = Visitors.IndexingVisitor()
newIndexer(instrument)
request  = newIndexer.getResults()

model = Atlas.SpotMarketDataModel(request, store) ## maybe add a setRequest method
marketData = model.marketData(evalDate)

newNPVVisitor = Visitors.NPVConstVisitor(marketData)
newNPVVisitor.visit(instrument)

newNPVVisitor.getResults()

Dual(955.989505)

In [112]:
for fx in marketData.fxs:
    print(fx)

print("")
for df in marketData.dfs:
    print(df)

Dual(0.001250)

Dual(1.000000)
Dual(0.994915)
Dual(0.989963)
Dual(0.984979)
Dual(0.980125)
Dual(0.975240)
Dual(0.970481)
Dual(0.965691)
Dual(0.960999)
Dual(0.956302)
Dual(0.951726)


In [113]:
newNotional = 1
instrument = Atlas.FixedRateBulletInstrument(startDate, endDate, paymentFrequency, newNotional, rate, discountContext)


instrument.currency(Atlas.USD())
instrument.discountCurveContext(store.curveContext("CLP"))
instrument.applyCcy(True)


newIndexer = Visitors.IndexingVisitor()
newIndexer(instrument)
request  = newIndexer.getResults()

model = Atlas.SpotMarketDataModel(request, store) ## maybe add a setRequest method
marketData = model.marketData(evalDate)

newNPVVisitor = Visitors.NPVConstVisitor(marketData)
newNPVVisitor.visit(instrument)

newNPVVisitor.getResults()

Dual(955.989505)